# Assignment 1

## Question 1: Think about a meaningful real-world application for this problem and briefly describe it

Application: Railway Scheduling and Track Layout

- Nodes in $U$ could represent fixed train stations along a route, where trains must stop in a specific order
- Nodes in $V$ could reoresent trains/train routes to be scheduled within the network
- Weighetd edges would inidcate the relationship between trains/train routes and tracks, with higher weights representing either higher traffic, longer distance, lower importance in terms of scheduling (in case of for exaple local vs. high speed train)
- constraints in C would enforce rules like the fact that certain trains need to arrive before others or that trains using the same tracks do not overlap.

The objective then would be to arrange the schedules in V to minimize the crossings of tracks while satisfying all contraints in C. This would lead to improved security and efficiency as minimzing track crossings would reduce the likelihood of collisions or delaysdue to track conflicts.

## Question 2: Develop a meaningful deterministic construction heuristic

A meaningful determinitic contruction heuristic could be one based on a greedy approach:
- Inititalize an empty ordered permuation list $\pi$ for nodes in $V$
- Compute for each node in $V$ the number of constraints that require other nodes to come before it
- Create a list of candidates with nodes in $V$ that have no predecessors

- For every node in the candidates and until $V$ is empty
  - calculate the total weight of edges connecting it to U
  - select the node with the lowest total edge weight to nodes in $U$ and append it to the final permutation list $\pi$

  - update the in-degree of nodes that had this as a constraint reducing it by 1

  - if the in-degree of any node in V reaches zero add it to the candidates




## Question 3: Derive a randomized construction heuristic to be applied iteratively

The randomized heuristic could follow the same process as the deterministic one, but then, when choosing the node in $V$ to add to the final permutation list $\pi$, instead of always choosing the one that has a minimum weighted sum of edges to $U$, pick one randomly, with a probability inversely propotional to this sum.

In [ ]:
def load_instance(filename):
    with open(filename, 'r') as file:
        U_size, V_size, C_size, E_size = map(int, file.readline().split())
        graph = Graph(U_size, V_size)

        # Read constraints section
        line = file.readline().strip()
        while line != "#constraints":
            line = file.readline().strip()

        for _ in range(C_size):
            v, v_prime = map(int, file.readline().split())
            graph.add_constraint(v, v_prime)

        # Read edges section
        line = file.readline().strip()
        while line != "#edges":
            line = file.readline().strip()

        for _ in range(E_size):
            u, v, weight = file.readline().split()
            graph.add_edge(int(u), int(v), float(weight))

    return graph

### Construction

In [ ]:
from collections import defaultdict, deque
import itertools

class Graph:
    def __init__(self, U_size, V_size):
        self.U = list(range(1, U_size + 1))  # Nodes in set U
        self.V = list(range(U_size + 1, U_size + V_size + 1))  # Nodes in fixed layer U
        self.edges = []  # List of edges (u, v, weight)
        self.constraints = defaultdict(list)  # Constraints as adjacency list for V
        self.in_degree = {v: 0 for v in self.V}   # Dictionary to store in-degree of nodes in V

    def add_node_U(self, node):
        self.U.append(node)

    def add_node_V(self, node):
        self.V.append(node)
        self.in_degree[node] = 0  # Initialize in-degree for nodes in V

    def add_edge(self, u, v, weight):
        self.edges.append((u, v, weight))

    def add_constraint(self, v1, v2):
        self.constraints[v1].append(v2)
        self.in_degree[v2] += 1  # Update in-degree due to precedence constraint

class MWCCPSolution:
    def __init__(self, graph):
        self.graph = graph
        self.pi = []  # This will store the final order of nodes in V

    def greedy_construction(self):
        # Initialize candidates with in-degree 0 nodes
        candidates = deque([v for v in self.graph.V if self.graph.in_degree[v] == 0])

        while candidates:
            # Select node with lowest edge weight to nodes in U
            best_node = None
            best_weight_sum = float('inf')  # Minimize total weight

            for v in candidates:
                # Calculate the sum of edge weights for the current candidate node
                weight_sum = sum(weight for u, v2, weight in self.graph.edges if v2 == v)
                if weight_sum < best_weight_sum:
                    best_weight_sum = weight_sum
                    best_node = v

            # Add the selected best_node to the pi
            self.pi.append(best_node)
            candidates.remove(best_node)

            # Update in-degrees based on the constraints, and add new candidates
            for v_next in self.graph.constraints[best_node]:
                self.graph.in_degree[v_next] -= 1
                if self.graph.in_degree[v_next] == 0:
                    candidates.append(v_next)

        return self.pi

    def calculate_cost(self):
        # Create a dictionary for quick lookup of node positions in the current ordering
        position = {node: idx for idx, node in enumerate(self.pi)}
        total_cost = 0

        # Iterate over all pairs of edges to count crossings
        for (u1, v1, w1), (u2, v2, w2) in itertools.combinations(self.graph.edges, 2):
            # Check if edges cross based on their positions
            if (u1 < u2 and position[v1] > position[v2]) or (u1 > u2 and position[v1] < position[v2]):
                # Add the sum of weights for the crossing edges to the total cost
                total_cost += w1 + w2

        return total_cost

### Tests

In [ ]:
# Example usage with a small problem instance
graph = Graph(5, 5)

# Adding edges with weights
graph.add_edge(1, 7, 1)
graph.add_edge(2, 8, 2)
graph.add_edge(3, 9, 1)
graph.add_edge(4, 10, 3)
graph.add_edge(5, 6, 11)

# Adding precedence constraints
graph.add_constraint(7, 10)
graph.add_constraint(8, 9)

# Create an MWCCP solution instance and use the greedy heuristic
solution = MWCCPSolution(graph)
ordering = solution.greedy_construction()


print("The initial ordering of nodes in V:", ordering)

print(solution.calculate_cost())

The initial ordering of nodes in V: [7, 8, 9, 10, 6]
0


In [ ]:
graph = load_instance('in.txt')

solution = MWCCPSolution(graph)
ordering = solution.greedy_construction()

print("The initial ordering of nodes in V:", ordering)

print(solution.calculate_cost())


The initial ordering of nodes in V: [35, 43, 39, 48, 27, 49, 26, 31, 30, 34, 37, 38, 45, 46, 28, 40, 33, 41, 32, 42, 36, 47, 50, 29, 44]
94914.0
